In [2]:
import dash
from dash import html, dcc, callback, Input, Output
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
import datetime

## Data ##
daily_noise = pd.read_csv("../Data/daily_noisedata_2022.csv")
monthly_noise = pd.read_csv("../Data/monthly_noisedata_2022.csv")

# Create dataframe with GPS coordinates
gps_data = {
    'description': ['MP 01: Naamsestraat 35  Maxim', 'MP 02: Naamsestraat 57 Xior', 'MP 03: Naamsestraat 62 Taste', 'MP 04: His & Hears', 'MP 05: Calvariekapel KU Leuven', 'MP 06: Parkstraat 2 La Filosovia', 'MP 07: Naamsestraat 81', 'MP08bis - Vrijthof'],
    'lat': [50.877121, 50.87650, 50.87590, 50.875237, 50.87452, 50.874078, 50.873808, 50.87873],
    'lon': [4.700708, 4.700632, 4.700262, 4.700071, 4.69985, 4.70005, 4.700007, 4.70115]
}
gps_df = pd.DataFrame(gps_data)

# Merging noise data with GPS coordinates
merged_daily = pd.merge(daily_noise, gps_df, on='description', how='left')
merged_monthly = pd.merge(monthly_noise, gps_df, on='description', how='left')

# Add a new column 'year' with value 2022 for all observations
merged_daily['year'] = 2022
# Create a new 'date' column by combining 'year', 'month', and 'day' but put the date in the European way
merged_daily['date'] = pd.to_datetime(merged_daily[['year', 'month', 'day']]).dt.strftime('%d %b %Y')

# Convert numeric month to month names (in a column date to use same name as above)
merged_monthly['date'] = pd.to_datetime(merged_monthly['month'], format='%m').dt.strftime('%B')

# Create a StandardScaler object
scaler = MinMaxScaler()
# Fit the StandardScaler to the column and transform the Lamax and laeq values
merged_daily['standardized_lamax'] = scaler.fit_transform(merged_daily[['lamax']])
merged_daily['standardized_laeq'] = scaler.fit_transform(merged_daily[['laeq']])
merged_monthly['standardized_lamax'] = scaler.fit_transform(merged_monthly[['lamax']])
merged_monthly['standardized_laeq'] = scaler.fit_transform(merged_monthly[['laeq']])


In [3]:
import plotly.graph_objects as go

# Create the Scattermapbox trace
data_trace = go.Scattermapbox(
    lat=merged_daily['lat'],
    lon=merged_daily['lon'],
    mode='markers',
    marker=dict(
        size=merged_daily['standardized_laeq'],
        sizemode='diameter',
        sizeref=0.1,
        color='blue'
    ),
    customdata=merged_daily[['date', 'description', 'laeq', 'standardized_laeq']],
    hoverlabel=dict(namelength=0),
    hovertemplate='Date: %{customdata[0]}<br>'
                  'Location: %{customdata[1]}<br>'
                  'Noise level: %{customdata[2]:.2f} dB(A)<br>'
                  'Standardized noise level: %{customdata[3]:.2f} dB(A)'
)

# Create the frames for animation
frames = []
for i, date in enumerate(merged_daily['date'].unique()):
    frame_data = go.Scattermapbox(
        lat=[merged_daily.loc[merged_daily['date'] == date, 'lat']],
        lon=[merged_daily.loc[merged_daily['date'] == date, 'lon']],
        mode='markers',
        marker=dict(
            size=merged_daily.loc[merged_daily['date'] == date, 'standardized_laeq'].tolist(),
            sizemode='diameter',
            sizeref=0.1,
            color='blue'
        ),
        customdata=[merged_daily.loc[merged_daily['date'] == date, ['date', 'description', 'laeq', 'standardized_laeq']]],
    )
    frame = go.Frame(data=[frame_data], name=f'day{i+1}')
    frames.append(frame)

# Set up the slider
slider = dict(
    active=0,
    steps=[dict(method='animate',
                args=[[f'day{i+1}'], dict(frame=dict(duration=300, redraw=True), fromcurrent=True, transition=dict(duration=0))],
                label=f'day{i+1}') for i in range(len(frames))],
)

# Create the layout
layout = go.Layout(
    mapbox=dict(
        center=dict(lat=50.876, lon=4.70020),
        zoom=15,
        style='open-street-map'
    ),
    height=800,
    width=800,
    margin=dict(l=20, r=400, t=0, b=100),
    updatemenus=[dict(type='buttons', showactive=False,
                      buttons=[dict(label='Play',
                                    method='animate',
                                    args=[None, dict(frame=dict(duration=300, redraw=True),
                                                     fromcurrent=True, transition=dict(duration=0),
                                                     easing='quadratic-in-out')])])],
)

# Create the figure
fig = go.Figure(data=[data_trace], layout=layout, frames=frames)
fig.update_layout(sliders=[slider])

# Show the figure
fig.show()
